In [2]:
import configparser

import altair as alt
import pandas as pd
import psycopg2

In [3]:
def format_cursor(cursor):
    desc = [desc[0] for desc in cursor.description]
    data_in_query = [{desc[j]: i[j] for j in range(len(desc))}
                      for i in cursor.fetchall()]

    return data_in_query

In [4]:
def data(data_in_query):
    df = pd.DataFrame(data_in_query)
    return df

In [5]:
config = configparser.ConfigParser()
config.read('TEST.ini', encoding="utf-8")

config = {i: config['postgresql'][i].replace('"', '')
           for i in config['postgresql']}

In [6]:
with psycopg2.connect(**config) as connect:
    with connect.cursor() as cursor_number_persons:
        cursor_number_persons.execute("""
SELECT date,
total_users,
new_users,
total_couriers,
new_couriers
FROM (
    SELECT COUNT(DISTINCT user_id) new_users,
    (SUM(COUNT(DISTINCT user_id)) OVER (ORDER BY date))::INTEGER total_users,
    date 
    FROM (
        SELECT DISTINCT user_id,
        date_trunc('day', MIN(time) OVER (PARTITION BY user_id ORDER BY order_id)) AS date
        FROM user_actions) t1
    GROUP BY date
    ORDER BY date
) users
JOIN (
    SELECT COUNT(DISTINCT courier_id) new_couriers,
    (SUM(COUNT(DISTINCT courier_id)) OVER (ORDER BY date))::INTEGER total_couriers,
    date
    FROM (
        SELECT DISTINCT courier_id,
            date_trunc('day', MIN(time) OVER (PARTITION BY courier_id ORDER BY courier_id)) AS date 
            FROM courier_actions
        ) t1
    GROUP BY date
    ORDER BY date
) couriers
USING(date)
        """)
        df_cursor_number_persons = data(format_cursor(cursor_number_persons))

print(df_cursor_number_persons.dtypes, sep="\n \n")

date              datetime64[ns]
total_users                int64
new_users                  int64
total_couriers             int64
new_couriers               int64
dtype: object


## СРАВНЕНИЕ ВСЕХ ПОЛЬЗОВАТЕЛЕЙ

In [7]:
colors = sorted(['#7fc97f', '#beaed4'])
color = iter(colors)

title = alt.TitleParams('Daily number of people', anchor='middle', font='Verdana', fontSize=14, color='darkred')

number_persons_date = alt.Chart(df_cursor_number_persons, title=title).encode(
    x='date'
)

line_users = number_persons_date.mark_line(
    color=str(next(color))
).encode(
    alt.Y('total_users')
)

text_users = line_users.mark_text(
    align='center',
    baseline='middle',
    dy=-20,
    color="darkred"
).encode(
    text="total_users:Q"
)

line_couriers = number_persons_date.mark_line(
    color=str(next(color))
).encode(
    alt.Y('total_couriers')
)

text_couriers = line_couriers.mark_text(
    align='center',
    baseline='middle',
    dy=-10,
    color="darkred"
).encode(
    text="total_couriers:Q"
)

number_persons = (line_users + text_users) + (line_couriers + text_couriers)

legend = alt.Chart(pd.DataFrame({'legend': ['total_users', 'total_couriers']})
).mark_point(size=200).encode(
    y=alt.Y('legend:N', axis=alt.Axis(title='')),
    color=alt.Color('legend:N', legend=None, scale=alt.Scale(range=sorted(colors, reverse=True))),
).properties(
    title='Total People Legend',
    height=60,
    width=40
)

number_persons = number_persons.properties(
    height=400,
    width=500
)

number_persons = alt.hconcat(number_persons, legend).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    labelFontSize=10.5,
    titleFontSize=12
)

In [8]:
display(number_persons)

c:\Users\Ferke\Desktop\Python Course Stepik\TEST Jupyter Python and SQL\venv\lib\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

## СРАВНЕНИЕ НОВЫХ ПОЛЬЗОВАТЕЛЕЙ

In [9]:
colors = sorted(['#7fc97f', '#beaed4'])
color = iter(colors)

title = alt.TitleParams('Daily number of new people', anchor='middle', font='Verdana', fontSize=14, color='darkred')

number_persons_date = alt.Chart(df_cursor_number_persons, title=title).encode(
    x='date'
)

line_users = number_persons_date.mark_line(
    color=str(next(color))
).encode(
    alt.Y('new_users')
)

text_users = line_users.mark_text(
    align='center',
    baseline='middle',
    dy=-20,
    color="darkred"
).encode(
    text="new_users:Q"
)

line_couriers = number_persons_date.mark_line(
    color=str(next(color))
).encode(
    alt.Y('new_couriers')
)

text_couriers = line_couriers.mark_text(
    align='center',
    baseline='middle',
    dy=-10,
    color="darkred"
).encode(
    text="new_couriers:Q"
)

number_persons = (line_users + text_users) + (line_couriers + text_couriers)

legend = alt.Chart(pd.DataFrame({'legend': ['new_couriers', 'new_users']})
).mark_point(size=200).encode(
    y=alt.Y('legend:N', axis=alt.Axis(title='')),
    color=alt.Color('legend:N', legend=None, scale=alt.Scale(range=sorted(colors, reverse=True))),
).properties(
    title='Total New People Legend',
    height=60,
    width=40
)

number_persons = number_persons.properties(
    height=400,
    width=500
)

number_persons = alt.hconcat(number_persons, legend).configure_axis(
    labelFont='Tahoma',
    titleFont='Verdana',
    labelFontSize=10.5,
    titleFontSize=12
)

In [10]:
display(number_persons)

c:\Users\Ferke\Desktop\Python Course Stepik\TEST Jupyter Python and SQL\venv\lib\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

## ПРИРОСТ ЧИСЛА ЛЮДЕЙ НА ПЛОЩАДКЕ

In [11]:
with psycopg2.connect(**config) as connect:
    with connect.cursor() as cursor_Relative_dynamics_people:
        cursor_Relative_dynamics_people.execute("""
SELECT date AS date,
new_users,
new_couriers,
total_couriers,
total_users,
ROUND((new_users::DECIMAL / LAG(new_users, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT new_users_change,
ROUND((new_couriers::DECIMAL / LAG(new_couriers, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT new_couriers_change,
ROUND((total_users::DECIMAL / LAG(total_users, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT total_users_growth,
ROUND((total_couriers::DECIMAL / LAG(total_couriers, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT total_couriers_growth
FROM (
    SELECT COUNT(DISTINCT user_id) new_users,
    (SUM(COUNT(DISTINCT user_id)) OVER (ORDER BY date))::INTEGER total_users,
    date 
    FROM (
        SELECT DISTINCT user_id,
        date_trunc('day', MIN(time) OVER (PARTITION BY user_id ORDER BY order_id)) AS date
        FROM user_actions) t1
    GROUP BY date
    ORDER BY date
) users
JOIN (
    SELECT COUNT(DISTINCT courier_id) new_couriers,
    (SUM(COUNT(DISTINCT courier_id)) OVER (ORDER BY date))::INTEGER total_couriers,
    date
    FROM (
        SELECT DISTINCT courier_id,
            date_trunc('day', MIN(time) OVER (PARTITION BY courier_id ORDER BY courier_id)) AS date 
            FROM courier_actions
        ) t1
    GROUP BY date
    ORDER BY date
) couriers
USING(date)""")
        df_cursor_Relative_dynamics_people = data(format_cursor(cursor_Relative_dynamics_people))

df_cursor_Relative_dynamics_people['date'] = (df_cursor_Relative_dynamics_people['date'].dt.strftime('%d/%m/%Y')).astype('string')

print(df_cursor_Relative_dynamics_people.dtypes, sep="\n \n")

date                     string[python]
new_users                         int64
new_couriers                      int64
total_couriers                    int64
total_users                       int64
new_users_change                float64
new_couriers_change             float64
total_users_growth              float64
total_couriers_growth           float64
dtype: object


In [12]:
columns_colors = {'new_users_change':'#7fc97f',
                'new_couriers_change':'#beaed4',
                'new_users_value':'darkred',
                'new_couriers_value':'darkblue'}

In [13]:
sorted_by_columns = dict(sorted(columns_colors.items()))

columns = list(sorted_by_columns)
colors = list(sorted_by_columns.values())



title = alt.TitleParams('Relative_dynamics_people', anchor='middle', font='Verdana', fontSize=14, color='darkred')

number_persons_date = alt.Chart(df_cursor_Relative_dynamics_people, title=title).encode(
    x='date'
)


line_users = number_persons_date.mark_bar(
    color=columns_colors['new_users_change']
).encode(
    alt.Y('new_users_change')
)


text_users = line_users.mark_text(
    align='center',
    baseline='middle',
    dy=0,
    color=columns_colors['new_users_value']
).encode(
    text="new_users_change:Q"
)


line_couriers = number_persons_date.mark_bar(
    color=columns_colors['new_couriers_change']
).encode(
    alt.Y('new_couriers_change')
)


text_couriers = line_couriers.mark_text(
    align='center',
    baseline='middle',
    dy=0,
    color=columns_colors['new_couriers_value']
).encode(
    text="new_couriers_change:Q"
)


number_persons = alt.layer(line_users, line_couriers, text_users, text_couriers)


legend = alt.Chart(pd.DataFrame({'legend': columns})
).mark_point(size=200).encode(
    y=alt.Y('legend:N', axis=alt.Axis(title='')),
    color=alt.Color('legend:N', legend=None, scale=alt.Scale(range=colors)),
).properties(
    title='Dynamics People Legend',
    height=60,
    width=40
)

number_persons = number_persons.properties(
    height=400,
    width=500
)

number_persons = alt.hconcat(number_persons, legend).configure_axis(
    labelFont='Verdana',
    titleFont='Verdana',
    labelFontSize=11,
    titleFontSize=12
)


In [14]:
display(number_persons)

c:\Users\Ferke\Desktop\Python Course Stepik\TEST Jupyter Python and SQL\venv\lib\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

### Другой варинт этого графика находится 
    в "ГЛАВА_2_1_Метрики_и_графики_(ПРОДОЛЖЕНИЕ).ipynb"